In [86]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import scipy.optimize as opt
from sklearn.linear_model import LogisticRegression

In [87]:
# Only look at rows with specialities that are "physician"-like
included_specialties = ['Internal Medicine', 'Anesthesiology', 'Family Practice', 'Chiropractic',
       'Obstetrics/Gynecology', 'Cardiac Surgery',
       'Cardiology', 'Dermatology',
       'Physical Medicine and Rehabilitation', 'Radiation Oncology',
       'Infectious Disease', 'Orthopedic Surgery', 'Endocrinology',
       'Urology', 'Emergency Medicine', 'Neurology', 'Nephrology',
       'Preventive Medicine', 'Hand Surgery', 'Pulmonary Disease',
       'Otolaryngology', 'Plastic and Reconstructive Surgery',
       'General Practice', 'Allergy/Immunology', 'Psychiatry',
       'Ophthalmology', 'Diagnostic Radiology', 'Psychiatry & Neurology',
       'General Surgery', 'Geriatric Medicine',
       'Gastroenterology', 'Thoracic Surgery', 'Neuropsychiatry',
       'Pain Management', 'Podiatry',
       'Hematology/Oncology', 'Optometry', 'Neurosurgery', 'Medical Oncology', 'Surgical Oncology',
       'Pediatric Medicine', 'Nuclear Medicine',
       'Naturopath', 'Osteopathic Manipulative Medicine',
       'Orthopaedic Surgery',
       'Family Medicine', 'Rheumatology',
       'Vascular Surgery',
       'Critical Care (Intensivists)', 'Hospitalist', 'Hematology', 'Maxillofacial Surgery',
       'Interventional Pain Management',
       'Oral & Maxillofacial Surgery', 'Optician',
       'Thoracic Surgery (Cardiothoracic Vascular Surgery)',
       'Neurological Surgery', 'Cardiac Electrophysiology',
       'Physical Medicine & Rehabilitation', 'Pathology',
       'Sports Medicine', 'Sleep Medicine',
       'Colorectal Surgery (formerly proctology)', 'Geriatric Psychiatry',
       'Addiction Medicine', 'Gynecological/Oncology',
       'Interventional Radiology', 'Peripheral Vascular Disease',
       'Plastic Surgery',
       'Interventional Cardiology', 'Prosthetist',
       'Hospice and Palliative Care',
       'Neuromusculoskeletal Medicine, Sports Medicine',
       'Colon & Rectal Surgery',
       'Radiology','Obstetrics & Gynecology',
       'Hospital (Dmercs Only)',
       'Medical Genetics',
       'Clinical Neuropsychologist', 'Naprapath','Pediatrics',
       'Audiologist (billing independently)', 'Phlebology']

included_set = set(included_specialties)

In [88]:
# Read the data !! Make sure `data.txt` is in the same folder as this notebook or point to it in the below line !!
df = pd.read_csv('data.txt', sep="	")

# Only grab providers who are individuals (as opposed to Organizations)
df = df[df.nppes_provider_first_name.notnull()]

# Only grab rows with valid PHYSICIAN specialties
df = df.loc[df.specialty_description.isin(included_set)]

# ^ In the above code, I only reassign to the variable `df` since as an after thought I wanted to remove those rows

all_unique_npi = df.npi.unique()
# Instead of the above line ^, if you want to use old data uncomment out the below lines
# old_X = pd.read_csv('features_combined.txt', sep="\t")
# old_X_without_column_0 = old_X.drop(old_X.columns[[0]], axis=1)
# old_y = pd.read_csv('labels_combined.txt', sep="\t")
# old_y_without_column_0 = old_y.drop(old_y.columns[[0]], axis=1)

# Grab unique npi
# int_version_npi = map(int, old_X_without_column_0.npi.unique())
# all_npi = df.npi.unique()
# set_of_unproccessed_npi = set(all_npi) - set(int_version_npi)
# all_unique_npi = np.asarray(list(set_of_unproccessed_npi))


In [98]:
# Create a map of index <=> specialty for label array creation
all_specialties = df.specialty_description.unique()
# Instead of the above line ^, if you want to use old data uncomment out the below lines for initial save
# and later usage (Do something similar for line `all_generic_names = df.generic_name.unique()`)
# all_specialties = pd.DataFrame(df.specialty_description.unique())
# all_specialties.to_csv("all_specialties.txt", sep='\t')
# all_specialties = pd.read_csv('all_specialties.txt', sep="\t")
# all_specialties = all_specialties.drop(all_specialties.columns[[0]], axis=1)
index_for_specialty = dict((specialty,i) for i, specialty in enumerate(all_specialties))
inv_map = {v: k for k, v in index_for_specialty.iteritems()}

# Create a map of generic_name => index for feature array creation
all_generic_names = df.generic_name.unique()
X_header = np.insert(all_generic_names, 0, 'npi')
feature_size = X_header.shape[0]
generic_name_to_index = {}
for i, generic_name in enumerate(X_header):
    if i == 0:
        continue
    generic_name_to_index[generic_name] = i

In [99]:
s0 = all_unique_npi[0:40000]
# With more time I would have figured out how to make my code run faster + grabbed all npi data, using something like this:
# s1 = all_unique_npi[100000:200000]
# s2 = all_unique_npi[200000:300000]
# s3 = all_unique_npi[300000:400000]
# s4 = all_unique_npi[400000:500000]
# s5 = all_unique_npi[500000:600000]
# s6 = all_unique_npi[600000:700000]
# s7 = all_unique_npi[700000:900000]
# Learned: You do not get errors if index is too big, interesting

In [100]:
X = pd.DataFrame(columns=X_header)
y = np.zeros(s0.shape[0])

In [ ]:
# CAUTION: This code takes a VERY long time to run.  On the order of an hour.
for i, npi in enumerate(s0):
    if (i % 500) == 0:
        print i # To see if anything is actually happening
    npi_matches = df.loc[df['npi'] == npi]

    speciality = npi_matches.specialty_description.values[0]
    y[i] = index_for_specialty[speciality]

    # We need to normalize bene_count across doctor's patient panel (as suggested by claims data)
    total_bene_counts = npi_matches.bene_count.fillna(5).sum()
    feature_vector = np.zeros(feature_size)
    feature_vector[0] = npi
    for generic_name in npi_matches.generic_name.unique():
        generic_name_matches = npi_matches.loc[npi_matches['generic_name'] == generic_name]
        index_for_feature_vector = generic_name_to_index[generic_name]
        normalized_bene_count = generic_name_matches.bene_count.fillna(5).sum() / total_bene_counts
        feature_vector[index_for_feature_vector] = normalized_bene_count
    X.loc[i] = feature_vector

In [1]:
# Save data for later usage without having to rebuild the features + labels
# https://stackoverflow.com/questions/16923281/pandas-writing-dataframe-to-csv-file
X.to_csv("features_combined.txt", sep='\t')
pd.DataFrame(y).to_csv("labels_combined.txt", sep='\t')


# If you end up wanting to append data sets together
# combined_X = old_X_without_column_0.append(X)
# combined_X.to_csv("features_combined2.txt", sep='\t')

# unraveled = old_y_without_column_0.values.ravel()
# combined_y = pd.DataFrame(unraveled + y)
# combined_y.to_csv("labels_combined2.txt", sep='\t')

In [120]:
Xtrain= X.head(32000)
Xtest = X.tail(8000)

Y = pd.DataFrame(y)
Ytrain = Y.head(32000)
Ytest = Y.tail(8000)

Xtrain_without_npi = Xtrain.drop(columns=['npi'])
Xtest_without_npi = Xtest.drop(columns=['npi'])

In [ ]:
# OR clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')
clf = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model = clf.fit(Xtrain_without_npi, Ytrain.values.ravel())

In [ ]:
# This code is for testing how reasonable it is that your test set will cover the things you want
ytrain_values = set()
for e in pd.DataFrame(Ytrain).values:
    ytrain_values.add(inv_map[e[0]])
    
ytest_values = set()
for e in pd.DataFrame(Ytest).values:
    ytest_values.add(inv_map[e[0]])
    
len(included_specialties), len(ytrain_values), len(ytest_values), len(ytrain_values.intersection(ytest_values))

In [112]:
Ytest_predict_proba = model.predict_proba(Xtest_without_npi)

In [113]:
Ytest_predict = model.predict(Xtest_without_npi)

In [ ]:
yravel = Ytest.values.ravel()
total_abs = len(yravel)
total_good = 0
good = set()
bad = set()
for i, e in enumerate(Ytest_predict):
    if e == yravel[i]:
        total_good = total_good + 1
        good.add(inv_map[e])
    bad.add((inv_map[e], inv_map[yravel[i]]))
    
print total_good, total_abs
# print good
# print bad

In [ ]:
Xtrain0 = X.head(4500)
Xtrain1 = X.tail(5500).head(4500)
Xtest = X.tail(5500).tail(1000)

Y = pd.DataFrame(y)
Ytrain0 = Y.head(4500)
Ytrain1 = Y.tail(5500).head(4500)
Ytest = Y.tail(5500).tail(1000)

Xtrain0_without_npi = Xtrain0.drop(columns=['npi'])
Xtrain1_without_npi = Xtrain1.drop(columns=['npi'])
Xtest_without_npi = Xtest.drop(columns=['npi'])

clf_without_extra_features = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model = clf_without_extra_features.fit(Xtrain0_without_npi, Ytrain0.values.ravel())

Ytrain1_predict_proba = model.predict_proba(Xtrain1_without_npi)

Xtrain1_without_npi.shape, pd.DataFrame(Ytrain1_predict_proba).shape

In [2]:
#In [3]:
#
#df1 = pd.DataFrame([1,2,3], index = np.arange(2000))
#df2 = pd.DataFrame([3,5,3], index = np.arange(2000))
#pd.concat([df1,df2], axis=1)
#Out[3]:
#   0  0
#2  1  3
#3  2  5
#4  3  3

#size = len(Ytest_predict_proba)
#df1 = pd.DataFrame(Ytest_predict_proba, index = np.arange(size))
#df2 = pd.DataFrame(Xtest_without_npi, index = np.arange(size))
#meow = pd.concat([df1,pd.DataFrame(df2)], axis=1)